<a href="https://colab.research.google.com/github/tgandor/urban_oculus/blob/master/colab/NewValidate_multiple_JPEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls

sample_data


In [0]:
import os

if not os.path.isdir('/content/gdrive/My Drive'):
    from google.colab import drive
    drive.mount('/content/gdrive')

!ls /content/gdrive/My\ Drive/results

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
20200516_121134_results
20200531_211211_results
20200531_211958_results
20200601_104032_results
20200601_203948_results
20200602_211120_x101_fpn
20200603_160513_x101_fpn
val2017_faster_rcnn_R_101_FPN_3x_q_100.json
val2017_faster_rcnn_R_101_FPN_3x_q_10.json
val2017_faster_rcnn_R_101_FPN_3x_q_14.json
val2017_faster_rcnn_R_101_FPN_3x_q_19.json
val2017_faster_rcnn_R_101_FPN_3x_q_1.json
val2017_faster_rcnn_R_101_FPN_3x_q_28.json
val2017_faster_rcnn_R_101_FPN_3x_q_

In [0]:
%%time
import os
if not os.path.isdir('urban_oculus'):
  !git clone https://github.com/tgandor/urban_oculus
!(cd urban_oculus; git pull)
%run urban_oculus/snippets/get_coco_val2017_nounzip.ipynb
!urban_oculus/snippets/install_detectron.sh
%run urban_oculus/snippets/validate_2017_functions.py

Cloning into 'urban_oculus'...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (487/487), done.
remote: Compressing objects: 100% (446/446), done.
remote: Total 487 (delta 153), reused 354 (delta 38), pack-reused 0
Receiving objects: 100% (487/487), 5.78 MiB | 9.60 MiB/s, done.
Resolving deltas: 100% (153/153), done.
Already up to date.
Archive:  annotations_trainval2017.zip
  inflating: datasets/coco/annotations/instances_train2017.json  
  inflating: datasets/coco/annotations/instances_val2017.json  
  inflating: datasets/coco/annotations/captions_train2017.json  
  inflating: datasets/coco/annotations/captions_val2017.json  
  inflating: datasets/coco/annotations/person_keypoints_train2017.json  
  inflating: datasets/coco/annotations/person_keypoints_val2017.json  
--2020-06-05 15:05:16--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.130.179
Connecting to images.cocodataset.org (images.coco

In [0]:
import time
time.strftime('%Y-%m-%d %H:%M:%S')
dir()


['COCOEvaluator',
 'DefaultPredictor',
 'In',
 'MODEL_ZOO_CONFIGS',
 'Out',
 '_',
 '_3',
 '_4',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__nonzero__',
 '__package__',
 '__spec__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'build_detection_test_loader',
 'detectron2',
 'drive',
 'exit',
 'filesizes',
 'get_cfg',
 'get_ipython',
 'inference_on_dataset',
 'json',
 'os',
 'quit',
 'result_name',
 'result_path',
 'run_validation',
 'save',
 'save_filesizes',
 'save_results',
 'shutil',
 'time',
 'tqdm']

In [0]:
def validate_coco2017(model_config, quality, output_dir):
  result_file = result_path(model_config, quality, output_dir)
  if os.path.exists(result_file):
    print(time.strftime('%Y-%m-%d %H:%M:%S'), 'result seems to exist:', result_file)
    print('skipping', model_config, quality)
    return

  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'unpacking')
  !unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
  # save_filesizes('filesizes_original.json', 'original', 0, output_dir)

  print(time.strftime('%Y-%m-%d %H:%M:%S'), f'degrading quality to {quality}')
  start = time.time()
  !mogrify -verbose -quality {quality} datasets/coco/val2017/*.jpg 2>&1 | tqdm --total 10000 >/dev/null
  elapsed = time.time() - start
  # save_filesizes(f'filesizes_{quality}.json', quality, elapsed, output_dir)

  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'finished', quality)
  results, inference_time =  run_validation(model_config)
  
  print(time.strftime('%Y-%m-%d %H:%M:%S'), 'finished', quality)
  save_results(model_config, quality, results, inference_time, output_dir)

In [0]:
%%time
# https://github.com/conansherry/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml
model_idx = 5
model_config = MODEL_ZOO_CONFIGS[model_idx]
output_dir = '/content/gdrive/My Drive/results'  # mounted or not, here I come
print('model config:', model_config)

for q in tqdm(range(5, 101, 9)):
  validate_coco2017(model_config, q, output_dir)

model config: COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml


2020-06-05 15:17:08 result seems to exist: /content/gdrive/My Drive/results/val2017_faster_rcnn_R_101_FPN_3x_q_5.json
skipping COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml 5
2020-06-05 15:17:08 result seems to exist: /content/gdrive/My Drive/results/val2017_faster_rcnn_R_101_FPN_3x_q_14.json
skipping COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml 14
2020-06-05 15:17:08 unpacking
5002it [00:06, 767.24it/s]              
2020-06-05 15:17:16 degrading quality to 23
100% 10000/10000 [00:52<00:00, 192.04it/s]
2020-06-05 15:18:10 finished 23


model_final_f6e8b1.pkl: 243MB [00:24, 10.1MB/s]                           


Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=17.72s).
Accumulating evaluation results...
DONE (t=2.47s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.281
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.423
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.306
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.409
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.240
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.334
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.338
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [0]:
import torch

print('Device count:', torch.cuda.device_count())

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3, 1), 'GB')

!head -n 11 /proc/cpuinfo


Device count: 1
Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
